In [1]:
import pandas

In [2]:
import random

In [3]:
from sklearn.neighbors import KNeighborsClassifier

In [4]:
abalones = pandas.read_excel("dataset_processado.xlsx");
test = pandas.read_excel("app_processado.xlsx");

accuracy = {}; #Grupo de parâmetros: acurácia

In [5]:
#Parâmetros de teste
numReps = 10;
numChanges = 1;
randomSplit = 0.2;

numPop = 100;
mutationRate = 0.02;
generations = 20;
population = {}

for n in range(numPop):
    population[n] = [0,0,0,0,0,0,0,0,3]

In [6]:
def generatePop():
    for m in range(numPop):
        newPop = [];
        for n in range(8):
            newPop.append(random.randrange(101))
        newPop.append(random.randrange(3,100))
        population[m] = newPop;

In [7]:
def checkFitness(weights):
    accuracyResults = [];
    for num in range(numReps):
        knn = KNeighborsClassifier(n_neighbors = weights[8]);

        #Selecionando índices para teste e treino
        trainingIndexes = [];
        testIndexes = [];
        n = 0;
        for item in abalones["sex"]:
            if random.random() > randomSplit:
                trainingIndexes.append(n)
            else:
                testIndexes.append(n)
            n+=1

        #Criando os dataframes de treino e teste e separando os classificadores
        testData = abalones.drop(trainingIndexes);
        testClasses = testData["type"];
        testData = testData.drop(columns=["type"]);

        trainingData = abalones.drop(testIndexes);
        classifications = trainingData["type"]
        trainingData = trainingData.drop(columns=["type"])

        #Aplicando pesos
        n = 0;
        for col in trainingData.columns:
            trainingData[col] = trainingData[col]*weights[n]
            testData[col] = testData[col]*weights[n]
            n+=1

        #Aplicando K-NN
        knn.fit(trainingData, classifications)

        results = knn.predict(testData);

        #Testando acurácia
        n = 0;
        correctPredictions = 0;
        for item in testClasses:
            if item == results[n]:
                correctPredictions += 1;
            n+=1  

        acc = correctPredictions/len(results)
        accuracyResults.append(acc)

    return sum(accuracyResults)/len(accuracyResults)
    '''print("Acurácia média em " + str(numReps) + " tentativas: " + str(meanAcc))
    print("\n----- PARÂMETROS -----\nNúmero de Vizinhos: " + str(weights[8])
          + "\nSexo: " + str(weights[0]) 
          + "\nComprimento: " + str(weights[1])
          + "\nDiâmetro: " + str(weights[2])
          + "\nAltura: " + str(weights[3])
          + "\nPeso cheio: " + str(weights[4])
          + "\nPeso vazio: " + str(weights[5])
          + "\nPeso das vísceras: " + str(weights[6])
          + "\nPeso da concha: " + str(weights[7]) + "\n\n"
         )'''

In [8]:
def getFitness():
    results = {};
    for n in range(numPop):
        results[n] = checkFitness(population[n]);
        #print ("Checando a adequação de "+ str(n+1) + " de " + str(numPop) )
    return results

In [9]:
def selection(fitness):
    #Normalizando a chance de reprodução de acordo com o fitness
    best = max(fitness.values())
    worst =  min(fitness.values())
    for k in fitness.keys():
        fitness[k] = (fitness[k]-worst) / (best-worst)
    
    #Agora, com a tabela normalizada, temos a probabilidade de reprodução entre 0 e 1. Enquanto o tamanho da população não for
    #50% do original, continuamos removendo

    removed = 0;
    tries = 0;
    #print(numPop/2)
    while removed < (numPop/2) and tries < 2000:
        ind = random.randrange(numPop);
        #print("Índice: " + str(ind) + "\nParâmetros: " + str(population[ind]));
        #Ignorar se já foi removido
        if population[ind] == 0:
            tries+=1
            continue;
        else:
            if random.random() > fitness[ind]:
                #print("População removida.")
                removed+=1
                #print("Já removidos: " + str(removed))
                population[ind] = 0;
            else:
                tries+=1
    


In [10]:
def mutate(individual):
    n = 0
    for gene in individual:
        if random.random() < mutationRate:
            individual[n] = random.randrange(101)
        n+=1;
    if individual[-1] <= 2:
        individual[-1] = 3;
    return individual;

In [11]:
def crossover():
    newborns = 0;
    waitlist = []
    while newborns < numPop/2:
        #Selecionar aleatoriamente dois indivíduos
        ind1 = random.randrange(numPop);
        ind2 = random.randrange(numPop);
        
        while population[ind1] == 0:
            ind1 = random.randrange(numPop);
        while population[ind2] == 0 or ind2 == ind1:
            ind2 = random.randrange(numPop);
            
        #Indivíduos selecionados e conferidos, agora é necessário definir o ponto de cruzamento
        crosspoint = random.randrange(9)
        
        #Agora ocorre o real cruzamento. Ind1 doa a primeira parte e Ind2 doa a segunda parte.
        newInd = []
        newInd.extend(population[ind1][0:crosspoint])
        newInd.extend(population[ind2][crosspoint:])
        
        #Com o indivíduo novo criado, ele é adicionado a uma lista de espera (para não participar dos cruzamentos), após ser mutado
        newInd = mutate(newInd)
        waitlist.append(newInd)
        newborns+=1
    
    #Agora que acabaram os cruzamentos, precisamos repopular.
    for item in waitlist:
        if 0 in list(population.values()):
            population[list(population.values()).index(0)] = item
        

In [12]:
def geneticAlg():
    generatePop();
    gen = 0;
    while gen < generations:
        print("Geração: " + str(gen))
        fitness = getFitness()
        print('Melhor acurácia: ' + str(max(list(fitness.values()))))
        selection(fitness)
        crossover()
        gen+=1

In [13]:
geneticAlg()

Geração: 0
Melhor acurácia: 0.6500559893473808
Geração: 1
Melhor acurácia: 0.6533814280588828
Geração: 2
Melhor acurácia: 0.6539493836079812
Geração: 3
Melhor acurácia: 0.6592095537989865
Geração: 4
Melhor acurácia: 0.6664517189121956
Geração: 5
Melhor acurácia: 0.6558074942395498
Geração: 6
Melhor acurácia: 0.6625636919259377
Geração: 7
Melhor acurácia: 0.6571106060286336
Geração: 8
Melhor acurácia: 0.6652704064611902
Geração: 9
Melhor acurácia: 0.6587188486324165
Geração: 10
Melhor acurácia: 0.6696236354907283
Geração: 11
Melhor acurácia: 0.6610220124721156
Geração: 12
Melhor acurácia: 0.6644314219324157
Geração: 13
Melhor acurácia: 0.6632770210097977
Geração: 14
Melhor acurácia: 0.6613901579161767
Geração: 15
Melhor acurácia: 0.6643529329627248
Geração: 16
Melhor acurácia: 0.6684752060057462
Geração: 17
Melhor acurácia: 0.6613364408315009
Geração: 18
Melhor acurácia: 0.6646634283053595
Geração: 19
Melhor acurácia: 0.6619948707153471


In [14]:
population

{0: [1, 31, 51, 49, 65, 63, 31, 69, 97],
 1: [1, 21, 29, 17, 30, 81, 9, 69, 90],
 2: [1, 31, 19, 21, 30, 63, 82, 72, 56],
 3: [1, 31, 35, 49, 65, 63, 22, 72, 56],
 4: [1, 31, 29, 91, 85, 84, 76, 81, 28],
 5: [1, 31, 29, 49, 65, 90, 9, 69, 90],
 6: [1, 31, 35, 49, 65, 63, 22, 69, 90],
 7: [1, 31, 29, 91, 85, 84, 9, 70, 28],
 8: [1, 39, 35, 17, 30, 81, 9, 69, 90],
 9: [1, 31, 19, 49, 30, 63, 9, 69, 97],
 10: [1, 31, 19, 49, 93, 85, 20, 94, 62],
 11: [1, 31, 51, 49, 93, 85, 9, 69, 90],
 12: [1, 7, 29, 49, 30, 63, 20, 94, 62],
 13: [1, 31, 19, 49, 93, 85, 9, 69, 90],
 14: [1, 7, 16, 53, 93, 85, 9, 70, 28],
 15: [1, 31, 51, 49, 93, 85, 9, 69, 90],
 16: [35, 31, 19, 49, 30, 63, 45, 70, 90],
 17: [1, 31, 29, 49, 65, 63, 9, 69, 97],
 18: [1, 7, 16, 53, 93, 85, 30, 93, 57],
 19: [1, 31, 35, 49, 30, 63, 9, 69, 97],
 20: [79, 7, 16, 53, 18, 87, 17, 94, 39],
 21: [1, 31, 19, 49, 30, 63, 22, 69, 90],
 22: [1, 31, 19, 49, 65, 63, 22, 72, 56],
 23: [1, 31, 29, 49, 65, 90, 9, 69, 90],
 24: [1, 31, 51,

In [15]:
fitness = getFitness()
print(fitness)

{0: 0.649142538729809, 1: 0.6503090719749488, 2: 0.6319266445112062, 3: 0.6502814424670579, 4: 0.6485240708646737, 5: 0.6540504886494124, 6: 0.6471400633438571, 7: 0.6603966764995033, 8: 0.6476947397483351, 9: 0.6547930438266848, 10: 0.6560403420094776, 11: 0.6424619886609004, 12: 0.6476536525490857, 13: 0.6522168064982095, 14: 0.653881552344307, 15: 0.6551753654868469, 16: 0.6274120112695953, 17: 0.6461242369392638, 18: 0.6530658129416644, 19: 0.6518720760891151, 20: 0.6366961566019393, 21: 0.663467037858738, 22: 0.6553101971131987, 23: 0.6512444958990135, 24: 0.6409700308641192, 25: 0.6366390419065346, 26: 0.6469520863679327, 27: 0.6498535658741107, 28: 0.6361027397435476, 29: 0.6454693189779095, 30: 0.641051331913277, 31: 0.6490134079546402, 32: 0.6523182454305486, 33: 0.643780797562277, 34: 0.6583819422586095, 35: 0.6534532133463433, 36: 0.6448176701638727, 37: 0.6448463599207104, 38: 0.6529669617903456, 39: 0.6538185294307128, 40: 0.6429426808904326, 41: 0.6469846066025735, 42: 0.

In [17]:
max(fitness.values())

0.6640421353284431

In [18]:
population[94]

[1, 7, 16, 53, 93, 85, 9, 69, 90]

In [19]:
w = population[94]

In [20]:
checkFitness(w)

0.6560040564976163

In [21]:
knn = KNeighborsClassifier(n_neighbors = w[8])

In [ ]:
knn.fit()

In [29]:
classes = abalones['type']
trainingData = abalones.drop(columns=['type'])
test = pandas.read_excel("app_processado.xlsx");

In [30]:
#Aplicando pesos
n = 0;
for col in trainingData.columns:
    trainingData[col] = trainingData[col]*w[n]
    test[col] = test[col]*w[n]
    n+=1

In [33]:
#Aplicando K-NN
knn.fit(trainingData, classes)

results = knn.predict(test);

In [34]:
results

array([3, 2, 2, ..., 1, 1, 2], dtype=int64)

In [37]:
# Enviando previsões realizadas com o modelo para o servidor
URL = "https://aydanomachado.com/mlclass/03_Validation.php"


#TODO Substituir pela sua chave aqui
DEV_KEY = "Ponte de Safena"

# json para ser enviado para o servidor
data = {'dev_key':DEV_KEY,
        'predictions':pandas.Series(results).to_json(orient='values')}

# Enviando requisição e salvando o objeto resposta
r = requests.post(url = URL, data = data)

# Extraindo e imprimindo o texto da resposta
pastebin_url = r.text
print(" - Resposta do servidor:\n", r.text, "\n")


In [38]:
data

{'dev_key': 'Ponte de Safena',
 'predictions': '[3,2,2,3,3,1,1,2,3,3,2,3,3,1,2,3,2,1,3,3,2,2,1,2,3,2,3,3,2,3,2,2,1,3,2,2,2,3,2,1,1,1,3,1,2,3,3,3,2,1,2,3,3,3,2,2,3,1,1,1,3,2,3,2,2,1,3,3,3,1,2,3,1,1,1,1,1,2,1,1,3,2,2,2,2,2,2,1,2,1,1,1,3,2,2,2,3,2,2,1,2,1,1,2,3,2,1,3,2,2,2,3,2,3,1,2,1,2,2,1,2,1,2,3,1,3,1,1,1,3,2,1,3,2,1,1,2,3,2,2,1,3,1,2,1,2,1,2,1,1,3,1,1,2,3,3,2,2,1,3,1,3,3,1,3,2,3,2,1,2,2,2,1,2,2,2,3,2,2,1,1,1,2,1,1,2,2,3,1,3,3,2,1,3,3,1,1,2,2,1,3,2,1,3,2,1,1,2,3,2,1,2,3,1,3,2,1,1,2,1,3,1,1,2,1,2,1,2,2,3,1,1,3,1,1,2,3,1,1,1,2,3,2,3,1,3,2,2,1,1,2,3,2,2,2,2,1,3,3,1,3,1,2,3,2,2,3,3,2,1,2,1,1,2,1,2,2,3,1,1,2,1,1,1,2,3,2,1,2,2,2,2,2,3,2,3,1,2,1,3,2,2,1,2,3,2,2,3,2,2,1,3,1,3,2,3,2,1,1,3,3,1,1,1,3,2,1,1,1,3,2,3,1,1,2,1,2,1,2,2,1,2,2,2,2,3,1,2,1,1,2,3,1,3,3,3,3,2,1,3,3,1,3,3,3,2,1,2,3,3,1,1,3,2,3,3,1,1,1,1,2,2,1,3,3,1,1,1,2,1,1,1,1,3,2,1,2,1,1,1,2,1,2,2,2,3,1,3,1,3,1,1,2,1,3,2,2,2,1,1,1,3,1,3,3,2,2,1,1,2,3,1,2,2,1,1,2,2,2,1,2,2,3,2,2,3,1,1,2,2,3,1,3,3,3,1,1,1,1,1,1,2,2,2,3,3,1,3,3,3,2,2,3,3,1,1

In [42]:
len([3,2,2,3,3,1,1,2,3,3,2,3,3,1,2,3,2,1,3,3,2,2,1,2,3,2,3,3,2,3,2,2,1,3,2,2,2,3,2,1,1,1,3,1,2,3,3,3,2,1,2,3,3,3,2,2,3,1,1,1,3,2,3,2,2,1,3,3,3,1,2,3,1,1,1,1,1,2,1,1,3,2,2,2,2,2,2,1,2,1,1,1,3,2,2,2,3,2,2,1,2,1,1,2,3,2,1,3,2,2,2,3,2,3,1,2,1,2,2,1,2,1,2,3,1,3,1,1,1,3,2,1,3,2,1,1,2,3,2,2,1,3,1,2,1,2,1,2,1,1,3,1,1,2,3,3,2,2,1,3,1,3,3,1,3,2,3,2,1,2,2,2,1,2,2,2,3,2,2,1,1,1,2,1,1,2,2,3,1,3,3,2,1,3,3,1,1,2,2,1,3,2,1,3,2,1,1,2,3,2,1,2,3,1,3,2,1,1,2,1,3,1,1,2,1,2,1,2,2,3,1,1,3,1,1,2,3,1,1,1,2,3,2,3,1,3,2,2,1,1,2,3,2,2,2,2,1,3,3,1,3,1,2,3,2,2,3,3,2,1,2,1,1,2,1,2,2,3,1,1,2,1,1,1,2,3,2,1,2,2,2,2,2,3,2,3,1,2,1,3,2,2,1,2,3,2,2,3,2,2,1,3,1,3,2,3,2,1,1,3,3,1,1,1,3,2,1,1,1,3,2,3,1,1,2,1,2,1,2,2,1,2,2,2,2,3,1,2,1,1,2,3,1,3,3,3,3,2,1,3,3,1,3,3,3,2,1,2,3,3,1,1,3,2,3,3,1,1,1,1,2,2,1,3,3,1,1,1,2,1,1,1,1,3,2,1,2,1,1,1,2,1,2,2,2,3,1,3,1,3,1,1,2,1,3,2,2,2,1,1,1,3,1,3,3,2,2,1,1,2,3,1,2,2,1,1,2,2,2,1,2,2,3,2,2,3,1,1,2,2,3,1,3,3,3,1,1,1,1,1,1,2,2,2,3,3,1,3,3,3,2,2,3,3,1,1,3,1,2,1,2,2,1,2,2,1,2,3,3,3,2,3,2,2,3,3,3,1,3,2,1,2,2,2,1,3,1,2,3,1,1,3,2,1,1,2,2,3,3,2,2,3,1,1,1,3,2,1,2,1,3,2,1,1,1,1,2,1,2,1,1,3,3,3,1,3,1,2,1,1,1,2,3,1,2,2,3,2,2,1,2,3,2,3,1,1,3,3,3,2,2,1,2,3,3,2,2,1,1,3,2,2,1,1,2,3,2,1,1,1,2,3,3,1,1,2,1,3,1,2,3,1,1,2,1,3,1,1,2,1,2,2,1,3,1,1,2,3,1,1,1,1,2,1,3,3,1,1,1,2,2,1,1,3,3,2,1,1,2,1,1,1,3,2,3,2,1,1,3,1,1,3,3,1,3,1,1,1,3,1,1,1,2,1,2,2,2,1,1,2,1,1,2,1,2,2,2,3,2,1,1,3,1,2,3,3,2,3,3,1,2,2,2,1,1,3,3,3,2,1,2,1,1,3,3,1,1,3,3,3,2,2,1,1,2,2,1,2,2,3,1,2,3,1,2,1,2,1,2,1,3,1,1,1,2,2,2,3,1,2,2,3,1,1,3,1,1,1,1,3,3,1,3,2,2,2,3,2,1,1,1,1,3,3,2,3,1,3,2,3,1,2,3,1,1,3,2,2,1,1,2,1,1,2,2,3,2,3,3,1,1,2,3,1,1,1,3,3,1,1,3,3,3,2,2,1,2,1,2,3,2,3,1,1,1,1,2,3,1,1,2,1,3,2,2,1,2,2,2,1,2,3,1,1,1,2,1,2,2,2,2,3,2,3,3,3,1,3,3,2,2,1,3,1,3,1,3,1,2,3,3,2,1,1,2,3,1,2,2,2,3,2,3,2,2,3,2,1,1,2,1,1,2,3,2,3,3,3,2,3,3,3,3,2,3,2,3,3,1,1,1,3,1,1,1,2,2,1,2,3,3,2,1,3,2,1,2,3,3,3,1,3,3,2,3,1,3,3,1,2,2,1,1,3,1,2,3,1,2,1,1,2,2,1,1,3,2,1,1,2,1,3,3,1,2,2,1,1,2,1,2,2,1,1,2,2,2,2,1,2,1,1,1,2,1,2,3,3,2,2,3,1,1,1,3,2,3,3,1,1,1,2,3,3,1,1,1,3,2,3,2,1,2,3,3,2,3,2,2,2,1,3,2,3,2,3,3,3,1,3,1,2,3,3,3,2,2,1,1,1,3,3,3,1,3,2,2,3,1,1,1,1,1,1,2])

1045

In [39]:
import requests

In [43]:
URL = "https://aydanomachado.com/mlclass/03_Validation.php"
r = requests.post(url = URL, data = data)
pastebin_url = r.text
print(" - Resposta do servidor:\n", r.text, "\n")


 - Resposta do servidor:
 {"status":"success","dev_key":"Ponte de Safena","accuracy":0.6095693779904306,"old_accuracy":0.35693779904306} 



In [ ]:
#Geração Aleatória
for n in range(numChanges):
    weights = [random.random(),random.random(),random.random(),random.random(),random.random(),random.random(),random.random(),random.random(), random.randrange(3, 50, 2)]    
    accuracyResults = [];

    for n in range(numReps):
        knn = KNeighborsClassifier(n_neighbors = weights[8]);

        #Selecionando índices para teste e treino
        trainingIndexes = [];
        testIndexes = [];
        n = 0;
        for item in abalones["sex"]:
            if random.random() > randomSplit:
                trainingIndexes.append(n)
            else:
                testIndexes.append(n)
            n+=1

        #Criando os dataframes de treino e teste e separando os classificadores
        testData = abalones.drop(trainingIndexes);
        testClasses = testData["type"];
        testData = testData.drop(columns=["type"]);

        trainingData = abalones.drop(testIndexes);
        classifications = trainingData["type"]
        trainingData = trainingData.drop(columns=["type"])

        #Aplicando pesos
        n = 0;
        for col in trainingData.columns:
            trainingData[col] = trainingData[col]*weights[n]
            testData[col] = testData[col]*weights[n]
            n+=1

        #Aplicando K-NN
        knn.fit(trainingData, classifications)

        results = knn.predict(testData);

        #Testando acurácia
        n = 0;
        correctPredictions = 0;
        for item in testClasses:
            if item == results[n]:
                correctPredictions += 1;
            n+=1  

        acc = correctPredictions/len(results)
        accuracyResults.append(acc)

    meanAcc = sum(accuracyResults)/len(accuracyResults)
    print("Acurácia média em " + str(numReps) + " tentativas: " + str(meanAcc))
    print("\n----- PARÂMETROS -----\nNúmero de Vizinhos: " + str(weights[8])
          + "\nSexo: " + str(weights[0]) 
          + "\nComprimento: " + str(weights[1])
          + "\nDiâmetro: " + str(weights[2])
          + "\nAltura: " + str(weights[3])
          + "\nPeso cheio: " + str(weights[4])
          + "\nPeso vazio: " + str(weights[5])
          + "\nPeso das vísceras: " + str(weights[6])
          + "\nPeso da concha: " + str(weights[7])
         )
    accuracy[str(weights)] = meanAcc

In [ ]:
#Forward selection

def forwardSelection(baseWeights, baseAcc):
    m = 0;
    accs = [];
    for col in abalones.columns[:8]:
        weights = baseWeights.copy();
        weights[m] = 1
        accuracyResults = [];
    
        for num in range(numReps):
            knn = KNeighborsClassifier(n_neighbors = weights[8]);

            #Selecionando índices para teste e treino
            trainingIndexes = [];
            testIndexes = [];
            n = 0;
            for item in abalones["sex"]:
                if random.random() > randomSplit:
                    trainingIndexes.append(n)
                else:
                    testIndexes.append(n)
                n+=1

            #Criando os dataframes de treino e teste e separando os classificadores
            testData = abalones.drop(trainingIndexes);
            testClasses = testData["type"];
            testData = testData.drop(columns=["type"]);

            trainingData = abalones.drop(testIndexes);
            classifications = trainingData["type"]
            trainingData = trainingData.drop(columns=["type"])

            #Aplicando pesos
            n = 0;
            for col in trainingData.columns:
                trainingData[col] = trainingData[col]*weights[n]
                testData[col] = testData[col]*weights[n]
                n+=1

            #Aplicando K-NN
            knn.fit(trainingData, classifications)

            results = knn.predict(testData);

            #Testando acurácia
            n = 0;
            correctPredictions = 0;
            for item in testClasses:
                if item == results[n]:
                    correctPredictions += 1;
                n+=1  

            acc = correctPredictions/len(results)
            accuracyResults.append(acc)

        meanAcc = sum(accuracyResults)/len(accuracyResults)
        '''print("Acurácia média em " + str(numReps) + " tentativas: " + str(meanAcc))
        print("\n----- PARÂMETROS -----\nNúmero de Vizinhos: " + str(weights[8])
              + "\nSexo: " + str(weights[0]) 
              + "\nComprimento: " + str(weights[1])
              + "\nDiâmetro: " + str(weights[2])
              + "\nAltura: " + str(weights[3])
              + "\nPeso cheio: " + str(weights[4])
              + "\nPeso vazio: " + str(weights[5])
              + "\nPeso das vísceras: " + str(weights[6])
              + "\nPeso da concha: " + str(weights[7]) + "\n\n"
             )'''
        accs.append(meanAcc)
        m+=1
    
    #print("Acurácias finais: " + str(accs) + '\n\n')
    
    #Índice do melhor resultado
    ind = accs.index(max(accs))
    
    #Conferindo se houve melhora nessa recursão, caso não, já retorna a última lista de parâmetros
    if accs[ind] <= baseAcc:
        print("Resultado final: " + str(accs[ind]) + "%")
        return baseWeights
    #Caso haja melhora, a função será chamada novamente, com uma nova base de pesos e com a melhor acurácia
    else:
        newWeights = baseWeights.copy();
        newWeights[ind] = 1;
        return forwardSelection(newWeights, accs[ind])
    
    
for neighs in [5,15,25,50]:
    print(forwardSelection([0,0,0,0,0,0,0,0,neighs], 0))

#bestSet = forwardSelection([0,0,0,0,0,0,0,0,5], 0)